In [2]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
import xgboost as xgb

from sklearn.model_selection import train_test_split


In [3]:
no_show_df = pd.read_csv("data/no_show_feature_engineered_no_extreme_locations.csv")
no_show_df.head()

,patientId,appointmentId,gender,scheduledDay,appointmentDay,age,neighborhood,scholarship,hypertension,diabetes,...,appointmentDayDOW__Monday,appointmentDayDOW__Saturday,appointmentDayDOW__Thursday,appointmentDayDOW__Tuesday,appointmentDayDOW__Wednesday,lat,lon,distanceFromCenter,distanceFromCenterLat,distanceFromCenterLon
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29 00:00:00,62,JARDIM DA PENHA,False,True,False,...,0,0,0,0,0,0.490525,-0.169426,-0.198132,-0.280227,-0.201355
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29 00:00:00,56,JARDIM DA PENHA,False,False,False,...,0,0,0,0,0,0.490525,-0.169426,-0.198132,-0.280227,-0.201355
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29 00:00:00,62,MATA DA PRAIA,False,False,False,...,0,0,0,0,0,0.788316,-0.166879,-0.190882,0.089317,-0.199223
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29 00:00:00,8,PONTAL DE CAMBURI,False,False,False,...,0,0,0,0,0,0.341636,-0.170699,-0.201469,-0.464992,-0.200074
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29 00:00:00,56,JARDIM DA PENHA,False,True,True,...,0,0,0,0,0,0.490525,-0.169426,-0.198132,-0.280227,-0.201355


- Change how gender is encoded
- Select features
- Define the baseline model
- Define xgboost model
- Define tensorflow model

In [5]:
# encode the gender as a binary
# NOTE: the gender didn't seem to affect no show by itself, but we're going to keep it and
#       and verify if the model can still use it in conjunction with other variables
no_show_df["isFemale"] = (no_show_df["gender"] == "F")*1

# select the columns that we want to keep
FEATURE_COLS = ["age","scholarship","hypertension","diabetes","alcoholism","handicap","smsSent",
                "daysInAdvance","lat","lon","isFemale","distanceFromCenterLat","scheduledDayHour"]
days_of_weeks_cols = [col_name for col_name in no_show_df.columns if "appointmentDayDOW__" in col_name]
days_of_weeks_cols

Index(['patientId', 'appointmentId', 'gender', 'scheduledDay',
       'appointmentDay', 'age', 'neighborhood', 'scholarship', 'hypertension',
       'diabetes', 'alcoholism', 'handicap', 'smsSent', 'noShow',
       'appointmentDayMonth', 'appointmentDayDayOfWeek',
       'appointmentDayMonthProgress', 'scheduledDayMonth',
       'scheduledDayDayOfWeek', 'scheduledDayMonthProgress',
       'scheduledDayHour', 'scheduledDayMinute', 'daysInAdvance',
       'appointmentDayDOW__Friday', 'appointmentDayDOW__Monday',
       'appointmentDayDOW__Saturday', 'appointmentDayDOW__Thursday',
       'appointmentDayDOW__Tuesday', 'appointmentDayDOW__Wednesday', 'lat',
       'lon', 'distanceFromCenter', 'distanceFromCenterLat',
       'distanceFromCenterLon', 'isFemale'],
      dtype='object')